In [1]:
import pandas as pd

In [3]:
df = pd.read_csv(r'C:\Users\pegas\OneDrive\Desktop\zip\2021\2021\jan\GFDLNFO_BACKADJUSTED_22012021.csv')

In [141]:
start_date = '2021-01-01'
end_date = '2021-03-30'
symbol = 'NIFTY'

In [140]:
import sqlite3
import pandas as pd

def fetch_data_and_filter(start_date, end_date, symbol):
    conn = sqlite3.connect(r'C:\Users\pegas\OneDrive\Desktop\zip\2021\index_data.db')
    c = conn.cursor()
    table_name = f'{symbol}_2021'
    query = f'''
            SELECT *
            FROM {table_name}
            WHERE timestamp BETWEEN ? AND ?
            '''
    c.execute(query, (start_date, end_date))
    data = c.fetchall()
    conn.close()
    
    df = pd.DataFrame(data)
    column_names = ["id", "timestamp", "symbol", "expiry", "type", "strike", "open", "high", "low", "close", "oi", "volume", "provider", "upload_time"]
    df.columns = column_names
    
    df = df[df['symbol'] == f'{symbol}-I.NFO']
    df = df[['timestamp', 'symbol', 'open' , 'high' , 'low', 'close']]
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df.sort_values(by='timestamp', inplace=True)
    df.reset_index(inplace=True, drop=True)

    return df


In [142]:
df = fetch_data_and_filter(start_date, end_date, symbol)

In [143]:
df

,timestamp,symbol,open,high,low,close
0,2021-01-04 09:15:00,NIFTY-I.NFO,14100.00,14170.00,14075.10,14118.60
1,2021-01-04 09:16:00,NIFTY-I.NFO,14119.95,14120.00,14112.05,14112.70
2,2021-01-04 09:17:00,NIFTY-I.NFO,14112.10,14116.70,14107.50,14111.10
3,2021-01-04 09:18:00,NIFTY-I.NFO,14111.60,14118.45,14111.20,14118.00
4,2021-01-04 09:19:00,NIFTY-I.NFO,14118.00,14124.00,14118.00,14124.00
...,...,...,...,...,...,...
20884,2021-03-26 15:25:00,NIFTY-I.NFO,14625.90,14632.05,14623.15,14631.75
20885,2021-03-26 15:26:00,NIFTY-I.NFO,14631.00,14632.50,14629.00,14632.00
20886,2021-03-26 15:27:00,NIFTY-I.NFO,14632.00,14633.15,14630.10,14632.00
20887,2021-03-26 15:28:00,NIFTY-I.NFO,14632.25,14632.95,14630.10,14630.50


In [95]:
import pandas_ta as ta

def calculate_rsi(df, period=14):
    close = df['close']
    rsi = ta.rsi(close, length=period)
    return rsi


In [97]:
def intraday_rsi_strategy(df):
    df['rsi'] = calculate_rsi(df)
    
    tradebook = []
    stop_loss = 45
    exit_time = pd.Timestamp(year=df['timestamp'].iloc[0].year,
                             month=df['timestamp'].iloc[0].month,
                             day=df['timestamp'].iloc[0].day,
                             hour=15,
                             minute=0)
    
    grouped = df.groupby(df['timestamp'].dt.date)
    
    for date, group in grouped:
        in_trade = False
        entry_time = None
        pnl = 0
        
        for index, row in group.iterrows():
            if not in_trade and row['rsi'] > 60:
                in_trade = True
                entry_time = row['timestamp']
                entry_price = row['close']
            elif in_trade and (row['rsi'] < stop_loss or row['timestamp'] == exit_time):
                in_trade = False
                exit_time = row['timestamp']
                exit_price = row['close']
                pnl = exit_price - entry_price
                tradebook.append({'entry_time': entry_time,
                                  'exit_time': exit_time,
                                  'pnl': pnl})
    
    return pd.DataFrame(tradebook)

# start_date = '2021-01-01'
# end_date = '2021-12-31'
# symbol = 'NIFTY'
# df = fetch_data_and_filter(start_date, end_date, symbol)

# Apply the strategy
tradebook = intraday_rsi_strategy(df)
print(tradebook)


             entry_time           exit_time     pnl
0   2021-02-01 09:55:00 2021-02-01 10:30:00   14.00
1   2021-02-01 10:40:00 2021-02-01 12:13:00   87.50
2   2021-02-01 12:50:00 2021-02-01 13:57:00  218.65
3   2021-02-02 09:15:00 2021-02-02 10:15:00  165.00
4   2021-02-02 11:01:00 2021-02-02 11:34:00  -28.60
..                  ...                 ...     ...
184 2021-03-26 10:40:00 2021-03-26 10:48:00  -36.00
185 2021-03-26 11:05:00 2021-03-26 11:21:00  -21.05
186 2021-03-26 11:28:00 2021-03-26 12:20:00   29.00
187 2021-03-26 12:40:00 2021-03-26 13:08:00  -20.80
188 2021-03-26 13:31:00 2021-03-26 14:15:00  -12.40

[189 rows x 3 columns]


In [100]:
df

,timestamp,symbol,open,high,low,close,rsi
0,2021-02-01 09:15:00,NIFTY-I.NFO,13785.00,13800.00,13742.55,13755.00,NaN
1,2021-02-01 09:16:00,NIFTY-I.NFO,13754.35,13768.50,13750.35,13764.60,NaN
2,2021-02-01 09:17:00,NIFTY-I.NFO,13764.50,13780.00,13764.30,13775.80,NaN
3,2021-02-01 09:18:00,NIFTY-I.NFO,13775.75,13783.80,13769.50,13775.55,NaN
4,2021-02-01 09:19:00,NIFTY-I.NFO,13777.95,13781.40,13755.05,13757.55,NaN
...,...,...,...,...,...,...,...
14495,2021-03-26 15:25:00,NIFTY-I.NFO,14625.90,14632.05,14623.15,14631.75,62.347082
14496,2021-03-26 15:26:00,NIFTY-I.NFO,14631.00,14632.50,14629.00,14632.00,62.458409
14497,2021-03-26 15:27:00,NIFTY-I.NFO,14632.00,14633.15,14630.10,14632.00,62.458409
14498,2021-03-26 15:28:00,NIFTY-I.NFO,14632.25,14632.95,14630.10,14630.50,61.199284


In [114]:
def moving_crossover_strategy(df, short_window=50, long_window=200):
    df['short_ma'] = df['close'].rolling(window=short_window, min_periods=1).mean()
    df['long_ma'] = df['close'].rolling(window=long_window, min_periods=1).mean()
    
    tradebook = []
    stop_loss = 45
    exit_time = pd.Timestamp(year=df['timestamp'].iloc[0].year,
                             month=df['timestamp'].iloc[0].month,
                             day=df['timestamp'].iloc[0].day,
                             hour=15,
                             minute=0)
    
    # Group by date
    grouped = df.groupby(df['timestamp'].dt.date)
    
    # Trading loop for each day
    for date, group in grouped:
        in_trade = False
        entry_time = None
        pnl = 0
        
        # Trading loop for the day
        for index, row in group.iterrows():
            if not in_trade and row['rsi'] > 60 and row['short_ma'] > row['long_ma']:
                in_trade = True
                entry_time = row['timestamp']
                entry_price = row['close']
            elif in_trade and (row['rsi'] < stop_loss or row['timestamp'] == exit_time or row['short_ma'] < row['long_ma']):
                in_trade = False
                exit_time = row['timestamp']
                exit_price = row['close']
                pnl = exit_price - entry_price
                tradebook.append({'entry_time': entry_time,
                                  'exit_time': exit_time,
                                  'pnl': pnl})
    
    return pd.DataFrame(tradebook)

# Apply the strategy
tradebook = moving_crossover_strategy(df)
print(original_tradebook)


KeyError: 'rsi'

In [146]:
import pandas as pd

def moving_crossover_strategy(df, short_window=50, long_window=200):
    df['short_ma'] = df['close'].rolling(window=short_window, min_periods=1).mean()
    df['long_ma'] = df['close'].rolling(window=long_window, min_periods=1).mean()
    
    tradebook = []
    stop_loss = 30
    exit_time = pd.Timestamp(year=df['timestamp'].iloc[0].year,
                             month=df['timestamp'].iloc[0].month,
                             day=df['timestamp'].iloc[0].day,
                             hour=15,
                             minute=0)
    
    grouped = df.groupby(df['timestamp'].dt.date)
    
    for date, group in grouped:
        in_trade = False
        entry_time = None
        pnl = 0
        trades_per_day = 0  
        
        for index, row in group.iterrows():
            if row['timestamp'].time() >= pd.Timestamp('09:30').time():
                if not in_trade and row['short_ma'] > row['long_ma']:
                    in_trade = True
                    entry_time = row['timestamp']
                    entry_price = row['close']
                    trades_per_day += 1  # Increment trades per day counter
                elif in_trade and (row['timestamp'] == exit_time or row['short_ma'] < row['long_ma'] or row['close'] < entry_price - stop_loss):
                    in_trade = False
                    exit_time = row['timestamp']
                    exit_price = row['close']
                    pnl = exit_price - entry_price
                    tradebook.append({'entry_time': entry_time,
                                      'exit_time': exit_time,
                                      'pnl': pnl,
                                      'trades_per_day': trades_per_day})  # Add trades per day to tradebook
    
    return pd.DataFrame(tradebook)


tradebook = moving_crossover_strategy(df,15,25)
print(tradebook)


             entry_time           exit_time    pnl  trades_per_day
0   2021-01-04 09:30:00 2021-01-04 09:51:00  -1.30               1
1   2021-01-04 09:59:00 2021-01-04 10:00:00   0.70               2
2   2021-01-04 10:30:00 2021-01-04 10:31:00  -3.00               3
3   2021-01-04 11:21:00 2021-01-04 11:58:00  32.60               4
4   2021-01-04 12:12:00 2021-01-04 13:05:00  29.95               5
..                  ...                 ...    ...             ...
489 2021-03-26 11:30:00 2021-03-26 12:11:00  16.45               4
490 2021-03-26 12:40:00 2021-03-26 13:10:00 -26.80               5
491 2021-03-26 13:33:00 2021-03-26 14:17:00  -6.05               6
492 2021-03-26 14:34:00 2021-03-26 14:41:00 -27.85               7
493 2021-03-26 15:01:00 2021-03-26 15:08:00  -3.65               8

[494 rows x 4 columns]


In [147]:
tradebook.pnl.cumsum()

0       -1.30
1       -0.60
2       -3.60
3       29.00
4       58.95
        ...  
489   -322.70
490   -349.50
491   -355.55
492   -383.40
493   -387.05
Name: pnl, Length: 494, dtype: float64

In [151]:
import concurrent.futures
import pandas as pd

def process_group(date, group, short_window, long_window):
    tradebook = []
    stop_loss = 30
    exit_time = pd.Timestamp(year=group['timestamp'].iloc[0].year,
                             month=group['timestamp'].iloc[0].month,
                             day=group['timestamp'].iloc[0].day,
                             hour=15,
                             minute=0)
    in_trade = False
    entry_time = None
    pnl = 0
    trades_per_day = 0

    for index, row in group.iterrows():
        if row['timestamp'].time() >= pd.Timestamp('09:30').time():
            if not in_trade and row['short_ma'] > row['long_ma']:
                in_trade = True
                entry_time = row['timestamp']
                entry_price = row['close']
                trades_per_day += 1
            elif in_trade and (row['timestamp'] == exit_time or row['short_ma'] < row['long_ma'] or row['close'] < entry_price - stop_loss):
                in_trade = False
                exit_time = row['timestamp']
                exit_price = row['close']
                pnl = exit_price - entry_price
                tradebook.append({'entry_time': entry_time,
                                  'exit_time': exit_time,
                                  'pnl': pnl,
                                  'trades_per_day': trades_per_day})

    return tradebook

def moving_crossover_strategy_threaded(df, short_window=50, long_window=200):
    df['short_ma'] = df['close'].rolling(window=short_window, min_periods=1).mean()
    df['long_ma'] = df['close'].rolling(window=long_window, min_periods=1).mean()

    grouped = df.groupby(df['timestamp'].dt.date)
    tradebooks = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        future_to_group = {executor.submit(process_group, date, group, short_window, long_window): (date, group) for date, group in grouped}
        for future in concurrent.futures.as_completed(future_to_group):
            date, group = future_to_group[future]
            try:
                tradebooks.append(future.result())
            except Exception as exc:
                print(f"Exception occurred for date {date}: {exc}")

    return pd.DataFrame([trade for trades in tradebooks for trade in trades])


tradebook = moving_crossover_strategy_threaded(df, 15, 25)
print(tradebook)


             entry_time           exit_time    pnl  trades_per_day
0   2021-01-04 09:30:00 2021-01-04 09:51:00  -1.30               1
1   2021-01-04 09:59:00 2021-01-04 10:00:00   0.70               2
2   2021-01-04 10:30:00 2021-01-04 10:31:00  -3.00               3
3   2021-01-04 11:21:00 2021-01-04 11:58:00  32.60               4
4   2021-01-04 12:12:00 2021-01-04 13:05:00  29.95               5
..                  ...                 ...    ...             ...
489 2021-03-26 11:30:00 2021-03-26 12:11:00  16.45               4
490 2021-03-26 12:40:00 2021-03-26 13:10:00 -26.80               5
491 2021-03-26 13:33:00 2021-03-26 14:17:00  -6.05               6
492 2021-03-26 14:34:00 2021-03-26 14:41:00 -27.85               7
493 2021-03-26 15:01:00 2021-03-26 15:08:00  -3.65               8

[494 rows x 4 columns]


In [152]:
tradebook.pnl.cumsum()

0       -1.30
1       -0.60
2       -3.60
3       29.00
4       58.95
        ...  
489   -322.70
490   -349.50
491   -355.55
492   -383.40
493   -387.05
Name: pnl, Length: 494, dtype: float64

In [150]:
tradebook.sort_values(['entry_time'])

,entry_time,exit_time,pnl,trades_per_day
72,2021-01-04 09:30:00,2021-01-04 09:51:00,-1.30,1
73,2021-01-04 09:59:00,2021-01-04 10:00:00,0.70,2
74,2021-01-04 10:30:00,2021-01-04 10:31:00,-3.00,3
75,2021-01-04 11:21:00,2021-01-04 11:58:00,32.60,4
76,2021-01-04 12:12:00,2021-01-04 13:05:00,29.95,5
...,...,...,...,...
475,2021-03-26 11:30:00,2021-03-26 12:11:00,16.45,4
476,2021-03-26 12:40:00,2021-03-26 13:10:00,-26.80,5
477,2021-03-26 13:33:00,2021-03-26 14:17:00,-6.05,6
478,2021-03-26 14:34:00,2021-03-26 14:41:00,-27.85,7


In [131]:
tradebook.pnl.cumsum()

0       -1.30
1       -0.60
2       -3.60
3       29.00
4       58.95
        ...  
489   -322.70
490   -349.50
491   -355.55
492   -383.40
493   -387.05
Name: pnl, Length: 494, dtype: float64

In [112]:
import concurrent.futures

def moving_crossover_strategy_threaded(df, short_window=50, long_window=200):
    def process_group(date, group):
        tradebook = []
        stop_loss = 45
        exit_time = pd.Timestamp(year=group['timestamp'].iloc[0].year,
                                 month=group['timestamp'].iloc[0].month,
                                 day=group['timestamp'].iloc[0].day,
                                 hour=15,
                                 minute=0)
        in_trade = False
        entry_time = None
        pnl = 0
        trades_per_day = 0

        # Trading loop for the day
        for index, row in group.iterrows():
            if not in_trade and row['rsi'] > 60 and row['short_ma'] > row['long_ma']:
                in_trade = True
                entry_time = row['timestamp']
                entry_price = row['close']
                trades_per_day += 1
            elif in_trade and (row['rsi'] < stop_loss or row['timestamp'] == exit_time or row['short_ma'] < row['long_ma']):
                in_trade = False
                exit_time = row['timestamp']
                exit_price = row['close']
                pnl = exit_price - entry_price
                tradebook.append({'entry_time': entry_time,
                                  'exit_time': exit_time,
                                  'pnl': pnl,
                                  'trades_per_day': trades_per_day})

        return tradebook

    # Calculate moving averages
    df['short_ma'] = df['close'].rolling(window=short_window, min_periods=1).mean()
    df['long_ma'] = df['close'].rolling(window=long_window, min_periods=1).mean()

    # Group by date
    grouped = df.groupby(df['timestamp'].dt.date)

    tradebooks = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        future_to_group = {executor.submit(process_group, date, group): (date, group) for date, group in grouped}
        for future in concurrent.futures.as_completed(future_to_group):
            date, group = future_to_group[future]
            try:
                tradebooks.append(future.result())
            except Exception as exc:
                print(f"Exception occurred for date {date}: {exc}")

    # Merge tradebooks
    merged_tradebook = pd.concat([pd.DataFrame(trades) for trades in tradebooks], ignore_index=True)
    return merged_tradebook

# Example usage:
# Assuming df is already fetched and filtered
# df = fetch_data_and_filter(start_date, end_date, symbol)

# Apply the strategy
tradebook = moving_crossover_strategy_threaded(df)
print(tradebook)


Exception occurred for date 2021-01-12: 'rsi'
Exception occurred for date 2021-01-07: 'rsi'
Exception occurred for date 2021-01-04: 'rsi'
Exception occurred for date 2021-01-14: 'rsi'
Exception occurred for date 2021-01-13: 'rsi'
Exception occurred for date 2021-01-18: 'rsi'
Exception occurred for date 2021-01-15: 'rsi'
Exception occurred for date 2021-01-20: 'rsi'
Exception occurred for date 2021-01-19: 'rsi'
Exception occurred for date 2021-01-05: 'rsi'
Exception occurred for date 2021-01-06: 'rsi'
Exception occurred for date 2021-01-08: 'rsi'
Exception occurred for date 2021-01-11: 'rsi'
Exception occurred for date 2021-01-25: 'rsi'
Exception occurred for date 2021-01-27: 'rsi'
Exception occurred for date 2021-01-28: 'rsi'
Exception occurred for date 2021-01-29: 'rsi'
Exception occurred for date 2021-02-01: 'rsi'
Exception occurred for date 2021-02-04: 'rsi'
Exception occurred for date 2021-02-05: 'rsi'
Exception occurred for date 2021-02-08: 'rsi'
Exception occurred for date 2021-0

ValueError: No objects to concatenate

In [89]:
df = fetch_data_and_filter(start_date, end_date, symbol)

In [90]:
df

,timestamp,symbol,open,high,low,close
0,2021-02-01 09:15:00,NIFTY-I.NFO,13785.00,13800.00,13742.55,13755.00
1,2021-02-01 09:16:00,NIFTY-I.NFO,13754.35,13768.50,13750.35,13764.60
2,2021-02-01 09:17:00,NIFTY-I.NFO,13764.50,13780.00,13764.30,13775.80
3,2021-02-01 09:18:00,NIFTY-I.NFO,13775.75,13783.80,13769.50,13775.55
4,2021-02-01 09:19:00,NIFTY-I.NFO,13777.95,13781.40,13755.05,13757.55
...,...,...,...,...,...,...
14495,2021-03-26 15:25:00,NIFTY-I.NFO,14625.90,14632.05,14623.15,14631.75
14496,2021-03-26 15:26:00,NIFTY-I.NFO,14631.00,14632.50,14629.00,14632.00
14497,2021-03-26 15:27:00,NIFTY-I.NFO,14632.00,14633.15,14630.10,14632.00
14498,2021-03-26 15:28:00,NIFTY-I.NFO,14632.25,14632.95,14630.10,14630.50


In [67]:
import pandas as pd

def simple_moving_average_crossover(df):
    df['SMA_short'] = df['close'].rolling(window=5).mean()
    df['SMA_long'] = df['close'].rolling(window=20).mean()
    
    tradebook = pd.DataFrame(columns=['timestamp', 'action', 'price'])
 
    for i in range(1, len(df)):
        if df['SMA_short'].iloc[i] > df['SMA_long'].iloc[i] and df['SMA_short'].iloc[i - 1] <= df['SMA_long'].iloc[i - 1]:
            tradebook = tradebook._append({'timestamp': df['timestamp'].iloc[i], 'action': 'BUY', 'price': df['close'].iloc[i]}, ignore_index=True)
        elif df['SMA_short'].iloc[i] < df['SMA_long'].iloc[i] and df['SMA_short'].iloc[i - 1] >= df['SMA_long'].iloc[i - 1]:
            tradebook = tradebook._append({'timestamp': df['timestamp'].iloc[i], 'action': 'SELL', 'price': df['close'].iloc[i]}, ignore_index=True)
    
    return tradebook
tradebook = simple_moving_average_crossover(df)
print(tradebook)


AttributeError: 'DataFrame' object has no attribute 'append'

In [68]:
def group_trading_hours(df):
    df = df[(df['timestamp'].dt.time >= pd.Timestamp('9:15').time()) & (df['timestamp'].dt.time <= pd.Timestamp('15:30').time())]
    grouped_df = df.groupby(df['timestamp'].dt.date)
    return grouped_df

In [70]:
grouped_df = group_trading_hours(df)

In [81]:
trade_log = pd.DataFrame(columns='trade_type,entry_time,entry_price,exit_time,exit_price'.split(','))


In [82]:
trade_log

,trade_type,entry_time,entry_price,exit_time,exit_price


In [83]:
trade_log = trade_log.append({'trade_type':'BUY',
                                      'entry_time':'1',
                                      'entry_price':'2'},ignore_index=True)

AttributeError: 'DataFrame' object has no attribute 'append'

In [85]:
trade_log = pd.DataFrame(columns=['trade_type', 'entry_time', 'entry_price'])

# Append a new row
new_row = {'trade_type':'BUY', 'entry_time':'1', 'entry_price':'2'}
trade_log = trade_log._append(new_row, ignore_index=True)

In [86]:
trade_log

,trade_type,entry_time,entry_price
0,BUY,1,2


In [15]:
column_names = ["id", "timestamp" ,"symbol", "expiry", "type", "strike", "open", "high", "low", "close", "oi", "volume", "provider", "upload_time"]

# Rename columns 
df.columns = column_names

,id,timestamp,symbol,expiry,type,strike,open,high,low,close,oi,volume,provider,upload_time
104357,104358,2021-02-01 09:15:00,NIFTY-I.NFO,I,FUT,None,13785.00,13800.00,13742.55,13755.00,9184875,427650,GFDLNFO_BACKADJUSTED_01022021.csv,2024-02-26 10:30:45.252588
104358,104359,2021-02-01 09:16:00,NIFTY-I.NFO,I,FUT,None,13754.35,13768.50,13750.35,13764.60,9184875,172125,GFDLNFO_BACKADJUSTED_01022021.csv,2024-02-26 10:30:45.252588
104359,104360,2021-02-01 09:17:00,NIFTY-I.NFO,I,FUT,None,13764.50,13780.00,13764.30,13775.80,9348975,142800,GFDLNFO_BACKADJUSTED_01022021.csv,2024-02-26 10:30:45.252588
104360,104361,2021-02-01 09:18:00,NIFTY-I.NFO,I,FUT,None,13775.75,13783.80,13769.50,13775.55,9348975,82350,GFDLNFO_BACKADJUSTED_01022021.csv,2024-02-26 10:30:45.252588
104361,104362,2021-02-01 09:19:00,NIFTY-I.NFO,I,FUT,None,13777.95,13781.40,13755.05,13757.55,9348975,98325,GFDLNFO_BACKADJUSTED_01022021.csv,2024-02-26 10:30:45.252588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
659288,659289,2021-02-09 15:26:00,NIFTY-I.NFO,I,FUT,None,15108.00,15109.80,15105.50,15105.50,11271300,53775,GFDLNFO_BACKADJUSTED_09022021.csv,2024-02-26 10:36:24.214858
659289,659290,2021-02-09 15:27:00,NIFTY-I.NFO,I,FUT,None,15105.05,15107.10,15104.00,15104.00,11266425,33375,GFDLNFO_BACKADJUSTED_09022021.csv,2024-02-26 10:36:24.214858
659290,659291,2021-02-09 15:28:00,NIFTY-I.NFO,I,FUT,None,15103.35,15106.50,15103.10,15103.75,11266425,35175,GFDLNFO_BACKADJUSTED_09022021.csv,2024-02-26 10:36:24.215849
659291,659292,2021-02-09 15:29:00,NIFTY-I.NFO,I,FUT,None,15103.30,15106.65,15101.30,15106.20,11266425,59475,GFDLNFO_BACKADJUSTED_09022021.csv,2024-02-26 10:36:24.215849
